# Idle (fridge side)

In [3]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output


def Pt1000_cal(r):
    if r < 25 or r > 1400:
        return 0
    coeffs = [
        -469.544790033,
        2142.429105073,
        -4278.355519358,
        4917.129912473,
        -3583.140137551,
        1717.059717072,
        -541.269035711,
        108.277676198,
        -12.478784052,
        0.631579503,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs))

def RuO2_10k_cal(r):
    r = r / 1000
    if r < 10.8 or r > 320:
        return 0
    coeffs = [
        6909.149278051,
        -38314.105969326,
        93795.121719782,
        -132881.190697614,
        120024.664196962,
        -71674.983797385,
        28300.34446649,
        -7125.704599868,
        1038.446472548,
        -66.75479913,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def RuO2_1k5_cal(r):
    if r < 2000 or r > 17000:
        return 0
    coeffs = [
        -109406225.453069,
        264570367.275206,
        -284130767.924156,
        177858095.325872,
        -71516430.0477208,
        19156216.6038666,
        -3418108.54721648,
        391778.170838493,
        -26174.3212975398,
        776.590777082747,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def TT1304_cal(r):
    if r < 1600 or r > 75000:
        return 0
    coeffs = [
        -85991.97200244224,
       170101.13604422542,
        -146816.39437206375,
        72250.85152375244,
        -22180.90228857295,
        4351.18582667978,
        -532.76537613975,
        37.23402757752,
        -1.13741001238,
        0,      
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000


def S0927_cal(r):
    if r < 600 or r > 100000:
        return 0
    coeffs = [
        340.0066359697,
        -428.9444332334,
        219.360664071,
        -56.2474119288,
        7.2087047046,
        -0.3691158405,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

_SENSORS = [
    ('3K_low', RuO2_10k_cal),      # 0
    ('Still', RuO2_10k_cal),       # 1
    ('50mK', RuO2_1k5_cal),        # 2
    ('MixingCh_low', TT1304_cal),  # 3
    ('MixingCh_high', Pt1000_cal), # 4
    ('Magnet', RuO2_10k_cal),      # 5
]

# _SENSORS = [
#     ('3K_high', RuO2_1k5_cal),      # 0
#     ('Still', RuO2_1k5_cal),       # 1
#     ('50mK', RuO2_1k5_cal),        # 2
#     ('MixingCh_low', TT1304_cal),  # 3
#     ('MixingCh_high', Pt1000_cal), # 4
#     ('Magnet', RuO2_1k5_cal),      # 5
# ]

def FridgeScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 0;LCH 5;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    for i in [int(0),int(1),int(2),int(3),int(4),int(5)]:
        avs.write(f'REM 1;SCP {i};EXC 3;SDY 5;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 0;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    for i in [int(0),int(1),int(2),int(3),int(4),int(5)]:
        avg, std = avs.query(f'REM 1;SCR {i};AVE?;STD?;REM 0').strip().split(';')
        avg = float(avg.split(' ')[-1])
        std = float(std.split(' ')[-1])
        yield i, avg, std


def print_stuff():
    channel = int(avs.query('MUX?').strip().split(' ')[1])
    
    avs.write('ADC')
    while int(avs.query('*OPC?').strip()) != 1:
        time.sleep(0.1)
    res = float(avs.query('RES?').strip().split(' ')[2])
    
    if int(avs.query('OVL?').strip().split(' ')[1]) != 0:
        return
    
    name, temp_cal = _SENSORS[channel]
    temp = temp_cal(res)
    print(f'Channel {channel}\t{name}\tResistance {res} Ω\tTemperature {temp:.4f} K')

    
while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB0::21::INSTR') as avs: # GPIB0::21 should be fridge side AVS
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = list(FridgeScan(avs))
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    print(time.strftime('%l:%M%p %Z on %b %d, %Y'))
    for chan, avg, std in values:
        name = _SENSORS[chan][0]
        temp = _SENSORS[chan][1](avg)
        print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
        errs = []
        with open(path+f'Fridge_{name}.txt', "a") as myfile: # Ensure File name matches AVS
            myfile.write(f'{t}\t{t2}\t{avg:>10.2f}\t{temp:>10.5f}\tIdle\t\n')
            myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(120)

 1:45PM PDT on Aug 10, 2023
         3K_low	  10010.80 Ω	   0.00000 K
          Still	  10069.80 Ω	   0.00000 K
           50mK	   1581.15 Ω	   0.00000 K
   MixingCh_low	   1496.15 Ω	   0.00000 K
  MixingCh_high	   1084.56 Ω	 295.75666 K
         Magnet	   9978.20 Ω	   0.00000 K
Scanned in 73.7 seconds.


KeyboardInterrupt: 

# march 2023 probe MXC high channel msmt

In [ ]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output


def Pt1000_cal(r):
    if r < 25 or r > 1400:
        return 0
    coeffs = [
        -469.544790033,
        2142.429105073,
        -4278.355519358,
        4917.129912473,
        -3583.140137551,
        1717.059717072,
        -541.269035711,
        108.277676198,
        -12.478784052,
        0.631579503,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs))

def RuO2_10k_cal(r):
    r = r / 1000
    if r < 10.8 or r > 320:
        return 0
    coeffs = [
        6909.149278051,
        -38314.105969326,
        93795.121719782,
        -132881.190697614,
        120024.664196962,
        -71674.983797385,
        28300.34446649,
        -7125.704599868,
        1038.446472548,
        -66.75479913,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def RuO2_1k5_cal(r):
    if r < 2000 or r > 17000:
        return 0
    coeffs = [
        -109406225.453069,
        264570367.275206,
        -284130767.924156,
        177858095.325872,
        -71516430.0477208,
        19156216.6038666,
        -3418108.54721648,
        391778.170838493,
        -26174.3212975398,
        776.590777082747,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def TT1304_cal(r):
    if r < 1600 or r > 75000:
        return 0
    coeffs = [
        -85991.97200244224,
       170101.13604422542,
        -146816.39437206375,
        72250.85152375244,
        -22180.90228857295,
        4351.18582667978,
        -532.76537613975,
        37.23402757752,
        -1.13741001238,
        0,      
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000


def S0927_cal(r):
    if r < 600 or r > 100000:
        return 0
    coeffs = [
        340.0066359697,
        -428.9444332334,
        219.360664071,
        -56.2474119288,
        7.2087047046,
        -0.3691158405,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

_SENSORS = [
    ('MixingCh_high', Pt1000_cal)
]

def FridgeScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 4;LCH 4;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    for i in [int(4)]:
        avs.write(f'REM 1;SCP {i};EXC 3;SDY 5;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 0;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    for i in [int(4)]:
        avg, std = avs.query(f'REM 1;SCR {i};AVE?;STD?;REM 0').strip().split(';')
        avg = float(avg.split(' ')[-1])
        std = float(std.split(' ')[-1])
        yield i, avg, std


def print_stuff():
    channel = int(avs.query('MUX?').strip().split(' ')[1])
    
    avs.write('ADC')
    while int(avs.query('*OPC?').strip()) != 1:
        time.sleep(0.1)
    res = float(avs.query('RES?').strip().split(' ')[2])
    
    if int(avs.query('OVL?').strip().split(' ')[1]) != 0:
        return
    
    name, temp_cal = _SENSORS[channel]
    temp = temp_cal(res)
    print(f'Channel {channel}\t{name}\tResistance {res} Ω\tTemperature {temp:.4f} K')

    
while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB4::20::INSTR') as avs:
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = list(FridgeScan(avs))
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    for chan, avg, std in values:
        name = _SENSORS[0][0]
        temp = _SENSORS[0][1](avg) 
        print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
        errs = []
        with open(path+f'Probe_{name}.txt', "a") as myfile:     
            myfile.write(f'{t}\t{t2}\t{avg:>10.2f}\t{temp:>10.5f}\tIdle\t\n')
            myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(10)

# march 2023 probe MXC low channel msmt

In [8]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output


def Pt1000_cal(r):
    if r < 25 or r > 1400:
        return 0
    coeffs = [
        -469.544790033,
        2142.429105073,
        -4278.355519358,
        4917.129912473,
        -3583.140137551,
        1717.059717072,
        -541.269035711,
        108.277676198,
        -12.478784052,
        0.631579503,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs))

def RuO2_10k_cal(r):
    r = r / 1000
    if r < 10.8 or r > 320:
        return 0
    coeffs = [
        6909.149278051,
        -38314.105969326,
        93795.121719782,
        -132881.190697614,
        120024.664196962,
        -71674.983797385,
        28300.34446649,
        -7125.704599868,
        1038.446472548,
        -66.75479913,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def RuO2_1k5_cal(r):
    if r < 2000 or r > 17000:
        return 0
    coeffs = [
        -109406225.453069,
        264570367.275206,
        -284130767.924156,
        177858095.325872,
        -71516430.0477208,
        19156216.6038666,
        -3418108.54721648,
        391778.170838493,
        -26174.3212975398,
        776.590777082747,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

def TT1304_cal(r):
    if r < 1600 or r > 75000:
        return 0
    coeffs = [
        -85991.97200244224,
       170101.13604422542,
        -146816.39437206375,
        72250.85152375244,
        -22180.90228857295,
        4351.18582667978,
        -532.76537613975,
        37.23402757752,
        -1.13741001238,
        0,      
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000


def S0927_cal(r):
    if r < 600 or r > 100000:
        return 0
    coeffs = [
        340.0066359697,
        -428.9444332334,
        219.360664071,
        -56.2474119288,
        7.2087047046,
        -0.3691158405,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000

_SENSORS = [
    
     ('MixingCh_low', TT1304_cal),
]

def FridgeScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 3;LCH 3;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    for i in [int(3)]:
        avs.write(f'REM 1;SCP {i};EXC 3;SDY 5;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 0;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    for i in [int(3)]:
        avg, std = avs.query(f'REM 1;SCR {i};AVE?;STD?;REM 0').strip().split(';')
        avg = float(avg.split(' ')[-1])
        std = float(std.split(' ')[-1])
        yield i, avg, std


def print_stuff():
    channel = int(avs.query('MUX?').strip().split(' ')[1])
    
    avs.write('ADC')
    while int(avs.query('*OPC?').strip()) != 1:
        time.sleep(0.1)
    res = float(avs.query('RES?').strip().split(' ')[2])
    
    if int(avs.query('OVL?').strip().split(' ')[1]) != 0:
        return
    
    name, temp_cal = _SENSORS[channel]
    temp = temp_cal(res)
    print(f'Channel {channel}\t{name}\tResistance {res} Ω\tTemperature {temp:.4f} K')

    
while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB4::20::INSTR') as avs:
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = list(FridgeScan(avs))
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    for chan, avg, std in values:
        name = _SENSORS[0][0]
        temp = _SENSORS[0][1](avg) 
        print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
        errs = []
        with open(path+f'Probe_{name}.txt', "a") as myfile:     
            myfile.write(f'{t}\t{t2}\t{avg:>10.2f}\t{temp:>10.5f}\tIdle\t\n')
            myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(10)

   MixingCh_low	   1531.46 Ω	   0.00000 K
Scanned in 28.6 seconds.
SCANNING
ID: *IDN PICOWATT,AVS47-IB,0,3.4


KeyboardInterrupt: 

# Janky Probe MXC temp for use while measuring 

In [5]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output

def TT1304_cal(r):
    if r < 1600 or r > 75000:
        return 0
    coeffs = [
        -85991.97200244224,
       170101.13604422542,
        -146816.39437206375,
        72250.85152375244,
        -22180.90228857295,
        4351.18582667978,
        -532.76537613975,
        37.23402757752,
        -1.13741001238,
        0,      
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000


_SENSORS = [('MixingCh_low', TT1304_cal)]
def MXCScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 3;LCH 3;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    avs.write(f'REM 1;SCP 3;EXC 3;SDY 1;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 0;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    avg = float(avs.query(f'REM 1;SCR 3;AVE?;REM 0').strip())
    #avg = float(avg.split(' ')[-1])
    yield avg


while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/igorTempFile"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB4::20::INSTR') as avs:
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = MXCScan(avs)
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    name = _SENSORS[0][0]
    temp = _SENSORS[0][1](avg) 
    print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
    errs = []
    with open(path+f'Fridge_{name}.txt', "w") as myfile:     
        myfile.write(f'{temp:>10.5f}')
        myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(5)

   MixingCh_low	   1903.10 Ω	   4.31831 K
Scanned in 17.9 seconds.
SCANNING
ID: *IDN PICOWATT,AVS47-IB,0,3.4


KeyboardInterrupt: 

In [9]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output

def Pt1000_cal(r):
    if r < 25 or r > 1400:
        return 0
    coeffs = [
        -469.544790033,
        2142.429105073,
        -4278.355519358,
        4917.129912473,
        -3583.140137551,
        1717.059717072,
        -541.269035711,
        108.277676198,
        -12.478784052,
        0.631579503,
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs))


_SENSORS = [('MixingCh_high', Pt1000_cal)]
def MXCScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 4;LCH 4;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    avs.write(f'REM 1;SCP 4;EXC 3;SDY 1;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 2;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    avg = float(avs.query(f'REM 1;SCR 3;AVE?;REM 0').strip())
    #avg = float(avg.split(' ')[-1])
    yield avg


while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB4::20::INSTR') as avs:
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = MXCScan(avs)
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    name = _SENSORS[0][0]
    temp = _SENSORS[0][1](avg) 
    print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
    errs = []
    with open(path+f'probe_{name}.txt', "w") as myfile:     
        myfile.write(f'{temp:>10.5f}')
        myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(10)

  MixingCh_high	  16212.30 Ω	   0.00000 K
Scanned in 0.5 seconds.


KeyboardInterrupt: 

In [2]:
with rm.open_resource('GPIB4::20::INSTR') as avs:
        avs.clear()
        print('ID:', avs.query('*IDN?').strip())

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [ ]:
import time
import pyvisa as visa

import numpy as np
from IPython.display import clear_output

def TT1304_cal(r):
    if r < 1600 or r > 75000:
        return 0
    coeffs = [
        -85991.97200244224,
       170101.13604422542,
        -146816.39437206375,
        72250.85152375244,
        -22180.90228857295,
        4351.18582667978,
        -532.76537613975,
        37.23402757752,
        -1.13741001238,
        0,      
    ]
    return np.power(10, np.polynomial.polynomial.polyval(np.log10(r), coeffs)) / 1000


_SENSORS = [('MixingCh_low', TT1304_cal)]
def MXCScan(avs):
    # Wait for it to finish whatever it was doing.
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Sweep settings.
    avs.write(f'REM 1;FCH 3;LCH 3;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    avs.write(f'REM 1;SCP 3;EXC 3;SDY 1;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    avs.write('REM 1;SCN 0;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    # Read out the results.
    avg = float(avs.query(f'REM 1;SCR 3;AVE?;REM 0').strip())
    #avg = float(avg.split(' ')[-1])
    yield avg


while True:
    rm = visa.ResourceManager()
    t = int(time.time())
    t2= time.ctime(t)
    path = "/Users/dgglab/Desktop/TempDataStorage/igorTempFile"
    try:
        print("SCANNING")
        with rm.open_resource('GPIB0::21::INSTR') as avs:
            avs.clear()
            print('ID:', avs.query('*IDN?').strip())
            values = MXCScan(avs)
            print(values)
    except Exception as e:
        print(e)
        time.sleep(10)
        continue
    clear_output(wait=True)
    name = _SENSORS[0][0]
    temp = _SENSORS[0][1](avg) 
    print(f'{name:>15}\t{avg:>10.2f} Ω\t{temp:>10.5f} K')
    errs = []
    with open(path+f'Fridge_{name}.txt', "w") as myfile:     
        myfile.write(f'{temp:>10.5f}')
        myfile.close()
    print(f'Scanned in {time.time() - t:.1f} seconds.')
    for chan, err in errs:
        print(f'{chan}: {err}')
    time.sleep(5)

In [14]:
import pyvisa

rm = visa.ResourceManager()
with rm.open_resource('GPIB0::21::INSTR') as avs:
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    avs.write(f'REM 1;MUX 5;REM 0')
    while avs.query('*OPC?').strip() != '1':
        time.sleep(1)
    channel = int(avs.query('MUX?').strip().split(' ')[1])
    print(channel)    
        "Macintosh HD:Users:dgglab:Desktop:control900:control:Release:temp.txt"

5


In [7]:
 print('ID:', avs.query('*IDN?').strip())

InvalidSession: Invalid session handle. The resource might be closed.

In [53]:
with rm.open_resource('GPIB0::21::INSTR') as avs:
    avs.clear()
    print(avs.query(f'REM 1;SCR {0};AVE?;STD?;REM 0').strip().split(';'))
    print(avs.query(f'RAN?').strip().split(';'))

['AVE  1.70278E+04', 'STD  8.5E+00']
['RAN 7']


In [9]:
path = "/Users/dgglab/Desktop/TempDataStorage"
name = 1
#for chan, avg, std in values:
with open(path+f'Channel{name}.dat', "a") as myfile:     
    myfile.write(f'\t{avg:>10.2f}\t{temp:>10.5f}\n'")
    myfile.close()
file1 = open(path+"/Users/dgglab/Desktop/Fridge.dat", "w")
file1.write("\n")
L = ["a \n", "b \n", "v"]
file1.writelines(L)
file1.close()

InvalidSession: Invalid session handle. The resource might be closed.

In [ ]:
S0927_cal(1200)

In [3]:
with rm.open_resource('GPIB0::21::INSTR') as avs:
    avs.clear()
    print('ID:', avs.query('*IDN?').strip())
    #avs.write('REM 1;FCH 0;LCH 1;SCI 600;ETC 0;TCP 30;ARN 1;REM 0')
    #for i in range(8):
    #    avs.write(f'REM 1;SCP {i};EXC 3;SDY 5;CNT 10;REM 0')
    # Start the sweep and wait for it to complete.
    #avs.write('REM 1;SCN 0;REM 0')
    #while avs.query('*OPC?').strip() != '1':
    #    time.sleep(1)
    for i in range(8):
        avg, std = avs.query(f'REM 1;SCR {i};AVE?;STD?;REM 0').strip().split(';')
        print(avg, std)
        avg = float(avg.split(' ')[-1])
        std = float(std.split(' ')[-1])
        print(avg, std)

ID: *IDN PICOWATT,AVS47-IB,0,3.3
AVE  STD


ValueError: could not convert string to float: 

In [ ]:
def print_stuff():
    channel = int(avs.query('MUX?').strip().split(' ')[1])
    
    avs.write('ADC')
    while int(avs.query('*OPC?').strip()) != 1:
        time.sleep(0.1)
    res = float(avs.query('RES?').strip().split(' ')[2])
    
    if int(avs.query('OVL?').strip().split(' ')[1]) != 0:
        return
    
    name, temp_cal = _SENSORS[channel]
    temp = temp_cal(res)
    print(f'Channel {channel}\t{name}\tResistance {res} Ω\tTemperature {temp:.4f} K')
